In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import cv2

In [48]:
path = 'D:\\ILBS_14_8_21_Sanjeev\\Work_to_be_Completed_ILBS\\'

In [49]:
pid = []
image_file = []
for fname in os.listdir(path):
    if fname.isdigit():
        pid.append(fname)
        for fname_ins in os.listdir(path+fname):
            image_file.append(path+fname+'\\'+fname_ins)

In [50]:
df_f = pd.DataFrame(image_file)

In [51]:
df_f[0][1].split('\\')[-2]

'00910'

In [52]:
df_label = pd.read_excel(path+'\\'+'HVPG value with CT images.xlsx')

In [53]:
df_label.head()

,ID,Date,HVPG
0,9109,2019-01-30,21
1,19699,2019-02-13,12
2,31322,2019-02-15,17
3,45341,2019-02-13,16
4,49565,2019-02-06,10


In [54]:
labels = []
for row in df_f.values.tolist():
    id_ = int(row[0].split('\\')[-2])
    if id_ == 910:
        id_ = 9109
    for rows in df_label.values.tolist():
        if id_==rows[0]:
            labels.append([row[0],rows[-1]])
           # print(label)
        #break
    #print(row,id_)
    #break

In [55]:
len(labels)

35011

In [56]:
len(df_f)

35011

In [57]:
df = pd.DataFrame(labels,columns=['path','label'])

In [58]:
df.head()

,path,label
0,D:\ILBS_14_8_21_Sanjeev\Work_to_be_Completed_I...,21
1,D:\ILBS_14_8_21_Sanjeev\Work_to_be_Completed_I...,21
2,D:\ILBS_14_8_21_Sanjeev\Work_to_be_Completed_I...,21
3,D:\ILBS_14_8_21_Sanjeev\Work_to_be_Completed_I...,21
4,D:\ILBS_14_8_21_Sanjeev\Work_to_be_Completed_I...,21


In [13]:

image_path = df['path'][0]
def imread(path):
    image_path = path
    IMG_HEIGHT = IMG_WIDTH =250
    image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
    image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image=np.array(image)
    image = image.astype('float32')
    image /= 255
    return image

In [14]:
df['imarray'] = df['path'].apply(imread)

In [59]:
from tensorflow import keras
#X = df['imarray'].values
y = np.array(df['label'])
y = keras.utils.to_categorical(y)

In [60]:
#sklearn image read
from skimage import io
all_images = []
IMG_HEIGHT = IMG_WIDTH =150
for row in df['path']:
    image= cv2.imread( row, cv2.IMREAD_GRAYSCALE)
    image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image=np.array(image)
    image = image.astype('float32')
    image /= 255
    #image = np.expand_dims(image, axis=0)
    all_images.append(image)

In [61]:
train_ = np.array(all_images)

In [62]:
train_.shape

(35011, 150, 150)

In [36]:
X_final = np.empty((35011,150,150))
for i in range(35011):
    X_final[i] = train_[i]

In [63]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_,y,test_size=0.1)


In [19]:
X_train.shape

(24507, 150, 150)

In [67]:
X_train_t = np.expand_dims(X_train,axis=3)
X_test_t = np.expand_dims(X_test, axis=3)

In [68]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization

In [78]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', 
                 input_shape=(150, 150,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(28, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 148, 148, 32)      320       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 146, 146, 64)      18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 341056)            0         
_________________________________________________________________
dense_14 (Dense)             (None, 128)               43655296  
_________________________________________________________________
dense_15 (Dense)             (None, 28)                3612      
Total params: 43,677,724
Trainable params: 43,677,724
Non-trainable params: 0
__________________________________________

In [ ]:
model.fit(X_train_t, y_train, batch_size=100, epochs=10, verbose=1)

Train on 31509 samples
